<a href="https://colab.research.google.com/github/joaomj/tipos_distancias/blob/master/tipos_distancias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Orientações gerais**
- Este código tem fins educacionais apenas.
- Você precisa criar um dataset de exemplo para testar esse código. Futuramente eu vou implementar essa função aqui.
- Cuidado com o excesso de requisições às APIs. Eu testei com 200 ceps distintos e demorei quase 2h para obter as distâncias.

# **Imports**

In [13]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
import time
import requests

In [7]:
# Configurações gerais

# determinando que ele não exiba os dados em formato notação científica
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Configuração para exibir apenas 2 dígitos após a vírgula
pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# **Funções**

In [8]:
# -----------------------------------------
# Função para completar CEPs com zeros à esquerda
def formata_cep(cep):
    return str(cep).zfill(8)


# -------------------------------------------

# Função para obter coordenadas de um CEP usando a API, com rate limiting
def get_coordinates(cep):
    url = f'https://cep.awesomeapi.com.br/json/{cep}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return float(data['lat']), float(data['lng'])
        else:
            return np.nan, np.nan
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {cep}: {e}")
        return np.nan, np.nan
    finally:
        # Esperar 1 segundo entre as requisições para não sobrecarregar a API
        time.sleep(1)

# Função para obter a distância de condução entre duas coordenadas com rate limit e cache
def get_driving_distance(origin_coords, dest_coords):

    # Declarar o cache de distâncias
    distance_cache = {}

    # Verificar se a distância já foi calculada e está no cache
    if (origin_coords, dest_coords) in distance_cache:
        return distance_cache[(origin_coords, dest_coords)]

    profile = "driving"  # Tipo de perfil, pode ser driving, walking, cycling etc.
    coordinates = f"{origin_coords[1]},{origin_coords[0]};{dest_coords[1]},{dest_coords[0]}"
    url = f"http://router.project-osrm.org/route/v1/{profile}/{coordinates}?overview=false&alternatives=false&steps=false&annotations=false"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'routes' in data and len(data['routes']) > 0:
            distance_km = data['routes'][0]['distance'] / 1000  # Convertendo de metros para quilômetros
            distance_cache[(origin_coords, dest_coords)] = distance_km
            time.sleep(1)  # Intervalo de 1 segundo entre as requisições
            return distance_km

    return np.nan

# **Loading data**

In [9]:
# Obtendo os dados diretamente do google drive

# Substitua pelo seu link de compartilhamento público do Google Drive - URL DE EXEMPLO
# public_url = 'https://drive.google.com/file/d/0000/view?usp=drive_link'

# Extraia o FILE_ID do link
file_id = public_url.split('/')[5]

# Crie o link para download
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Use pandas para ler o CSV
df = pd.read_csv(download_url)

# Exibir as primeiras linhas do DataFrame
df.head()


,idShop,distinctOrigins,sku,quantity,formatted_date,displayName,isRecommendation,estimatedDeliveryTimeValue,estimatedDeliveryTimeUnit,shippingMethod,...,descriptionLogistic,deliveryMethodType,deliveryEstimateBusinessDays,originBranchOfficeId,origin,destination,costOfGoods,productCategory,weight,providerShippingCost
0,idShop_0,1,sku_0,1,2024-04-15 14:01:09,displayName_0,isRecommendation_0,7,DAY,shippingMethod_0,...,descriptionLogistic_0,deliveryMethodType_0,7.00,originBranchOfficeId_0,3110010,13560470,6.29,productCategory_0,0.016,7.03
1,idShop_0,1,sku_0,1,2024-04-15 14:01:09,displayName_1,isRecommendation_0,3,DAY,shippingMethod_1,...,descriptionLogistic_1,deliveryMethodType_1,3.00,originBranchOfficeId_0,3110010,13560470,6.29,productCategory_0,0.016,16.37
2,idShop_0,1,sku_0,1,2024-04-15 14:01:09,displayName_2,isRecommendation_0,0,DAY,shippingMethod_2,...,descriptionLogistic_2,deliveryMethodType_2,0.00,originBranchOfficeId_0,3110010,3110010,6.29,productCategory_0,0.016,0.00
3,idShop_1,1,sku_0,1,2024-04-15 14:01:14,displayName_0,isRecommendation_0,7,DAY,shippingMethod_0,...,descriptionLogistic_0,deliveryMethodType_0,7.00,originBranchOfficeId_0,3110010,13560470,9.99,productCategory_0,0.016,7.07
4,idShop_1,1,sku_0,1,2024-04-15 14:01:14,displayName_1,isRecommendation_0,3,DAY,shippingMethod_1,...,descriptionLogistic_1,deliveryMethodType_1,3.00,originBranchOfficeId_0,3110010,13560470,9.99,productCategory_0,0.016,16.37


In [ ]:
# Caso o arquivo esteja baixado no seu diretório:

# Obtendo o path deste notebook
current_dir = os.getcwd()

# Importando arquivos
df = pd.read_csv('dados.csv') # nome fictício, você precisa criar sua base
# df. head()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6789 entries, 0 to 6788
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   idShop                        6789 non-null   object 
 1   distinctOrigins               6789 non-null   int64  
 2   sku                           6789 non-null   object 
 3   quantity                      6789 non-null   int64  
 4   formatted_date                6789 non-null   object 
 5   displayName                   6789 non-null   object 
 6   isRecommendation              6789 non-null   object 
 7   estimatedDeliveryTimeValue    6789 non-null   int64  
 8   estimatedDeliveryTimeUnit     6789 non-null   object 
 9   shippingMethod                6789 non-null   object 
 10  fulfillmentMethod             6789 non-null   object 
 11  state                         6789 non-null   object 
 12  freightCostCurrency           6789 non-null   object 
 13  des

# **Obtendo as distâncias**
Campos 'origin' e 'destination' são CEPs. Alguns ceps que deveriam começar com zero estão sem esses números, sendo necessário formatá-los para que todos os ceps tenham 8 dígitos.

In [11]:
df['origin'].nunique()

30

In [12]:
df['destination'].nunique()

125

In [ ]:
# colunas 'origin' e 'destination' têm os CEPs
unique_origins = df['origin'].unique()
unique_destinations = df['destination'].unique()

# Dicionário para armazenar as distâncias entre os CEPs
cep_distances = {}

# Iterar sobre os pares únicos de origem e destino
for origin_cep in unique_origins:
    # Obter coordenadas do CEP de origem
    origin_coords = get_coordinates(origin_cep)
    if np.isnan(origin_coords).any():
        continue  # Pular CEPs sem coordenadas válidas

    for dest_cep in unique_destinations:
        # Obter coordenadas do CEP de destino
        dest_coords = get_coordinates(dest_cep)
        if np.isnan(dest_coords).any():
            continue  # Pular CEPs sem coordenadas válidas

        # Calcular a distância de condução entre as coordenadas
        distance = get_driving_distance(origin_coords, dest_coords)

        # Armazenar a distância no dicionário
        cep_distances[(origin_cep, dest_cep)] = distance
